# Ajuste de Hiper Parâmetros:

- Spark facilita o teste de diferentes hiper parâmetros
- você pode testar diferentes configurações de todo o pipeline
- ParamGridBuilder: configura quais parâmetros e quais domínios quer testar
- Utiliza cross-validation ou train-validation split

# Cross-Validation:
- É uma técnica de tuning de hiper parâmetros
- Divide os dados de treino em partições (folds) que são usadas para treino (2/3 dos dados) e 
teste (1/3 dos dados)
- Qtd partições = Qtd conjunto de treino = Qtd conjunto de teste
- Partições são treinadas e testadas nos domínios do grid de hiper parâmetros fornecido
- Quando encontrar o melhor conjunto de hiper parâmetros, ele novamente recria o modelo
usando todos os dados de treino

# Train-Validation Split:

- Testa cada configuração do grid de hiper parâmetros uma única vez
- trainingrati define a proporção usada para treino e teste
- Quando encontrar o melhor conjunto de hiper parâmetros, ele novamente recria o modelo
usando todos os dados de treino

# Comparativo:
![title](CrossValid_TrainValid.png) 

In [1]:
import findspark, pyspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName("TunningTrain").getOrCreate()

In [2]:
iris = spark.read.csv("iris.csv", header=True, inferSchema=True, sep=",")
irisTreino, irisTeste = iris.randomSplit([0.7,0.3])

In [3]:
from pyspark.ml.feature import VectorAssembler
vector = VectorAssembler (inputCols=["sepallength","sepalwidth","petallength","petalwidth"], 
                          outputCol = "independente")

In [4]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="class", outputCol="label")

In [5]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
mlp = MultilayerPerceptronClassifier(maxIter=100, layers=[4,5,4,3], featuresCol="independente",
                                    labelCol="label")

In [6]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vector, indexer, mlp])

In [8]:
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
performance = MulticlassClassificationEvaluator(labelCol="label", metricName="accuracy")
grid = ParamGridBuilder().addGrid(mlp.maxIter,[10,100,1000]).addGrid(mlp.layers,[[4,5,4,3],[4,4,3]]).build()
trainval = TrainValidationSplit(estimator=pipeline,estimatorParamMaps=grid, evaluator=performance,
                         trainRatio=0.8)

In [10]:
modelo = trainval.fit(irisTreino)

In [11]:
previsao = modelo.transform(irisTeste)
previsao.select("rawprediction","probability","prediction","label").show()

+--------------------+--------------------+----------+-----+
|       rawprediction|         probability|prediction|label|
+--------------------+--------------------+----------+-----+
|[9.81516203619439...|[0.99996792649168...|       0.0|  0.0|
|[9.76460118723611...|[0.99996590401996...|       0.0|  0.0|
|[9.78052668579353...|[0.99996655421834...|       0.0|  0.0|
|[9.79471211484377...|[0.99996712302200...|       0.0|  0.0|
|[9.80158829408027...|[0.99996739537577...|       0.0|  0.0|
|[9.73985045227198...|[0.99996486723146...|       0.0|  0.0|
|[9.81079540378062...|[0.99996775640033...|       0.0|  0.0|
|[9.82030800135405...|[0.99996812530159...|       0.0|  0.0|
|[1.60997042417048...|[0.61241910641459...|       0.0|  2.0|
|[9.81126324333367...|[0.99996777461702...|       0.0|  0.0|
|[9.81864233420977...|[0.99996806099927...|       0.0|  0.0|
|[9.81720004802964...|[0.99996800522710...|       0.0|  0.0|
|[9.81949433254978...|[0.99996809390757...|       0.0|  0.0|
|[9.81551267338560...|[0

In [12]:
performance = MulticlassClassificationEvaluator(labelCol="label",predictionCol="prediction", 
                                                metricName="accuracy")
acuracia = performance.evaluate(previsao)
print(acuracia)

0.9565217391304348


In [ ]:
# Tentativa de extrair quais foram os melhores hiperparâmetros do Grid:

In [101]:
melhormodelo = modelo.bestModel

In [62]:
#Printa atributos de um objeto:
print(dir(melhormodelo))

['__abstractmethods__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_copyValues', '_copy_params', '_defaultParamMap', '_dummy', '_from_java', '_is_protocol', '_paramMap', '_params', '_randomUID', '_resetUid', '_resolveParam', '_set', '_setDefault', '_shouldOwn', '_testOwnParam', '_to_java', '_transform', 'clear', 'copy', 'explainParam', 'explainParams', 'extractParamMap', 'getOrDefault', 'getParam', 'hasDefault', 'hasParam', 'isDefined', 'isSet', 'load', 'params', 'read', 'save', 'set', 'stages', 'transform', 'uid', 'write']


In [102]:
print(dir(melhormodelo.stages[-1]._java_obj))

['$', '$anonfun$setThresholds$1', '$anonfun$transform$1', '$anonfun$transform$2', '$anonfun$transform$2$adapted', '$anonfun$transform$3', '$anonfun$transform$3$adapted', '$anonfun$transform$4', '$anonfun$transform$5', '$anonfun$transform$5$adapted', '$anonfun$transform$6', '$anonfun$transform$6$adapted', '$anonfun$transform$7', '$anonfun$transform$7$adapted', '$anonfun$transform$8', '$anonfun$transformImpl$1', '$anonfun$transformImpl$1$adapted', '$anonfun$transformSchema$1', '$anonfun$transformSchema$2', 'blockSize', 'clear', 'copy', 'copyValues', 'copyValues$default$2', 'defaultCopy', 'defaultParamMap', 'equals', 'evaluate', 'explainParam', 'explainParams', 'extractParamMap', 'featuresCol', 'featuresDataType', 'findSummaryModel', 'get', 'getBlockSize', 'getClass', 'getDefault', 'getFeaturesCol', 'getInitialWeights', 'getLabelCol', 'getLayers', 'getMaxIter', 'getOrDefault', 'getParam', 'getPredictionCol', 'getProbabilityCol', 'getRawPredictionCol', 'getSeed', 'getSolver', 'getStepSize'

In [105]:
print(melhormodelo.stages[1]._java_obj.getMaxIter)

In [106]:
print(melhormodelo.stages[1]._java_obj.maxIterParam)

In [107]:
print(type(melhormodelo.stages[1]._java_obj.getMaxIter))

<class 'py4j.java_gateway.JavaMember'>


In [111]:
print(melhormodelo.stages[1]._java_obj.getMaxIter.name)

getMaxIter


In [113]:
print(melhormodelo.stages[1]._java_obj.getMaxIter.__getattribute__)

<method-wrapper '__getattribute__' of JavaMember object at 0x0000023CC6B76FD0>


In [112]:
print(dir(melhormodelo.stages[1]._java_obj.getMaxIter.__getattribute__))

['__call__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__name__', '__ne__', '__new__', '__objclass__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__self__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__text_signature__']
